# Object detection

This notebook explores different methods of simple object detection in printed forms using OpenCV. Object detection serves as one of the methods of digitising printed forms, so the data can be used for further analysis. The accompanying blog post can be found [here](https://fuzzylabs.ai/blog/simple-object-detection/)

As an example, we will take a section of a questionnaire and try to detect checkboxes present in it. The sample image can be found at `gs://fuzzylabs-jupyter-delicacies/form_segment.png`.

This approach can be extended to account for other objects on forms (so these forms can be digitised for the purpose of further data analysis). Therefore, you are encouraged to try it on different images (local files and Google Cloud Storage are supported) and modify the code to make it work on your examples.

Run this cell to install the necessary packages, if they are not already installed.

In [ ]:
!pip install opencv-python numpy matplotlib gcsfs

In [ ]:
from cv2 import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import gcsfs

In [ ]:
def show_image(img, colorspace=cv2.COLOR_GRAY2RGB):
    plt.figure(figsize=(12,12))
    plt.imshow(cv2.cvtColor(img, colorspace))
    plt.show()
    
def imread_wrapper(uri):
    if uri.startswith("gs://"):
        with gcsfs.GCSFileSystem().open(uri, "rb") as f:
            arr = np.asarray(bytearray(f.read()), dtype="uint8")
            img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
            return img
    else: # Assume local file
        return cv2.imread(uri, cv2.IMREAD_COLOR)

In [ ]:
img = imread_wrapper("gs://fuzzylabs-jupyter-delicacies/form_segment.png")
show_image(img, colorspace=cv2.COLOR_BGR2RGB)

# Image preparation

Three methods of preprocessing are investigated:
* Thresholding (with Otsu's method to determine an optimal threshold)
* Opening on the thresholded image. Modifiable parameter: `KERNEL_LENGTH`
* Canny edge detector. Modifiable parameter: `CANNY_KERNEL_SIZE`

In [ ]:
# Image thresholding
_, img_bin = cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
img_bin = 255 - img_bin
print("Thresholded image")
show_image(img_bin)

# Opening with vertical kernel
KERNEL_LENGTH = 15
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,KERNEL_LENGTH))
vertical = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, vertical_kernel, iterations=1)
print("Vertical opened image")
show_image(vertical)
# Opening with horizontal kernel 
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (KERNEL_LENGTH,1))
horizontal = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
print("Horizontal opened image")
show_image(horizontal)
# Combining openings
img_opened = cv2.addWeighted(vertical, 0.5, horizontal, 0.5, 0.0)
_, img_opened = cv2.threshold(img_opened, 0, 255, cv2.THRESH_BINARY)
print("Combined opened image")
show_image(img_opened)

#
CANNY_KERNEL_SIZE = 50
print("Canny edge detection")
img_canny = cv2.Canny(img, CANNY_KERNEL_SIZE, CANNY_KERNEL_SIZE)
show_image(img_canny, cv2.COLOR_BGR2RGB)

# Contours

In [ ]:
def draw_contours(contours):
    show_image(cv2.drawContours(img.copy(), contours, -1, (0, 255, 0), 1), cv2.COLOR_BGR2RGB)

def get_contours(img_bin):
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    draw_contours(contours)
    return contours

print("Thresholded image")
thresholded_contours = get_contours(img_bin)
print("Opened image")
opened_contours = get_contours(img_opened)
print("Canny edge detection")
edged_contours = get_contours(img_canny)

# Contour Filtering
Multiple methods to filter the resulting contours

In [ ]:
def filter_contours(contours, func):
    _contours = [x for x in contours if func(x)]
    draw_contours(_contours)

Filter by contour area, parameters: `expected_area` and `tolerance`

In [ ]:
def is_correct_area(contour, expected_area=625, tolerance=100):
    area = cv2.contourArea(contour)
    return abs(area - expected_area) <= tolerance

print("Thresholded image")
filter_contours(thresholded_contours, is_correct_area)
print("Opened image")
filter_contours(opened_contours, is_correct_area)
print("Canny edge detection")
filter_contours(edged_contours, is_correct_area)

Evidently, this filtering is not robust enough. We also want only the square boxes to be detected

Filtering by assessing bounding boxes for squareness. Parameters: `expected_area`, `tolerance` and `squareness_tolerance`

In [ ]:
def are_bounding_dimensions_correct(contour, expected_area=625, tolerance=100, squareness_tolerance=5):
    area = cv2.contourArea(contour)
    x, y, w, h = cv2.boundingRect(contour)
    return abs(area - expected_area) <= tolerance and abs(w - h) <= squareness_tolerance

print("Thresholded image")
filter_contours(thresholded_contours, are_bounding_dimensions_correct)
print("Opened image")
filter_contours(opened_contours, are_bounding_dimensions_correct)
print("Canny edge detection")
filter_contours(edged_contours, are_bounding_dimensions_correct)

The previous method still picks up the circle as a valid check box

Filtering using shape matching, parameter -- `tolerance`

The `template` can be modified to catch shapes different from a 25x25 square.

In [ ]:
def does_contour_match(contour, tolerance=0.0055):
    template = np.array([[[0, 0]], [[0, 25]], [[25, 25]], [[25, 0]]], dtype=np.int32)
    return cv2.matchShapes(template, contour, 1, 0.0) <= tolerance

print("Thresholded image")
filter_contours(thresholded_contours, does_contour_match)
print("Opened image")
filter_contours(opened_contours, does_contour_match)
print("Canny edge detection")
filter_contours(edged_contours, does_contour_match)